### Bill Cleanup + Other Data Wrangling
- This notebook specifically pulls all data that needs to be trimmed and sorted (and deleted if need be)
- It takes in 2009-2022 California Legislature Bills from leginfo.ca.gov as csv
- It merges all bills into a single csv

- As of right now, the term sorting method is limited due to low ram on my computer + I haven't optimized the web scraping portion yet
    - In response, the bills are removed based on the title/descriptions taken from leginfo itself

- The final result is a large csv of California Legislature Bills from a certain time period only related to Law Enforcement or any other search terms provided
    - This might open doors for context analysis for other types of bills!

In [102]:
#Only pandas is used for this, everything else is extra just in case
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from bs4 import BeautifulSoup # Beautiful Soup library used for web scraping
import requests # this is needed to communicate with the html file


In [103]:
#Writing all of the dataframes into memory to be combined into one

raw_2009_2010 = pd.read_csv("raw_bill_data/bills2009_2010.csv")
raw_2011_2012 = pd.read_csv("raw_bill_data/bills2011_2012.csv")
raw_2013_2014 = pd.read_csv("raw_bill_data/bills2013_2014.csv")
raw_2015_2016 = pd.read_csv("raw_bill_data/bills2015_2016.csv")
raw_2017_2018 = pd.read_csv("raw_bill_data/bills2017_2018.csv")
raw_2019_2020 = pd.read_csv("raw_bill_data/bills2019_2020.csv")
raw_2021_2022 = pd.read_csv("raw_bill_data/bills2021_2022.csv")

In [104]:
#Creating finished dataframe starting with first set of data
Full_Raw_DataFrame = raw_2009_2010.copy()
#Doing the rest of the data
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2011_2012)
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2013_2014)
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2015_2016)
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2017_2018)
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2019_2020)
Full_Raw_DataFrame = Full_Raw_DataFrame.append(raw_2021_2022)

Full_Raw_DataFrame = Full_Raw_DataFrame.reset_index(drop=True)


In [105]:
#Dropping nans due to only 2 of them existing
Full_Raw_DataFrame.dropna(subset=['status_date'], inplace = True)
Full_Raw_DataFrame.dropna(subset=['last_action_date'], inplace = True)
Full_Raw_DataFrame = Full_Raw_DataFrame.reset_index(drop=True) #Resetting index as loop uses it

index=0 #Index counter

for row in Full_Raw_DataFrame['status_date']:
    row= row.replace('-','',)
    Full_Raw_DataFrame.at[index,'status_date'] = row
#Rewrites the row under the 'Status' column so that the new string is there
    index += 1
index=0
for row in Full_Raw_DataFrame['last_action_date']:
    row= row.replace('-','',)
    Full_Raw_DataFrame.at[index,'last_action_date'] = row
#Rewrites the row under the 'Status' column so that the new string is there
    index += 1
Full_Raw_DataFrame

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
0,59869,30,AB1,5,Vetoed,20091011,Teachers: program of professional growth: conf...,An act to amend Section 44277 of the Education...,0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB1/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
1,59870,30,AB2,5,Vetoed,20091011,Individual health care coverage.,"An act to add Sections 1389.9, 1389.10, 1389.1...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB2/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
2,59871,30,AB3,5,Vetoed,20091011,Workforce development: Renewable Energy Workforce,"An act to amend Sections 14202, 14203, 14204, ...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB3/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
3,59872,30,AB4,1,Introduced,20081201,Constitutional convention: delegates.,An act relating to the Constitutional Convention.,0,NaN,20100202,From committee: Filed with the Chief Clerk pur...,https://legiscan.com/CA/bill/AB4/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
4,59873,30,AB5,4,Passed,20090629,Civil discovery: Electronic Discovery Act.,"An act to amend Sections 2016.020, 2031.010, 2...",0,NaN,20090629,"Chaptered by Secretary of State - Chapter 5, S...",https://legiscan.com/CA/bill/AB5/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33653,1385441,1791,SCA1,1,Introduced,20201207,Elections: referenda.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210407,Referred to Com. on E. & C.A.,https://legiscan.com/CA/bill/SCA1/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33654,1385541,1791,SCA2,1,Introduced,20201207,Public housing projects.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210429,From committee: Be adopted and re-refer to Com...,https://legiscan.com/CA/bill/SCA2/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33655,1385494,1791,SCA3,1,Introduced,20201207,Elections: recalls.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210407,Referred to Com. on E. & C.A.,https://legiscan.com/CA/bill/SCA3/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33656,1455043,1791,SCA4,1,Introduced,20210217,Legislature: 2-year budget.,A resolution to propose to the people of the S...,476,Senate Budget and Fiscal Review,20210407,Referred to Coms. on B. & F.R. and E. & C.A.,https://legiscan.com/CA/bill/SCA4/2021,http://leginfo.legislature.ca.gov/faces/billSt...


In [106]:
Full_Raw_DataFrame = Full_Raw_DataFrame.reset_index(drop=True) #Resetting index

Full_Raw_DataFrame.to_csv("bills.csv")

In [107]:
#Keyforstatusdesc = {Vetoed:00, Introduced:01, Passed:10, Engrossed:11, Enrolled:4, Failed:5, Failed:6}
#df_codes = pd.read_csv('https://raw.githubusercontent.com/lupedeanda/test_repo/main/codes.csv')
#Took unique words from each code
final_list_of_words = ['confidential', 'squad', 'vicinity', 'identity', 'dispatching',
                     'misdemeanor', 'warrant', 'negative', 'fight', 'officer', 'duty',
                     'reckless', 'operation', 'pursuit', 'emergency', 'units', 'service',
                     'ambush', 'condition', 'patient', 'case', 'intoxicated', 'urgent',
                     'subject', 'traffic', 'complaint', 'standby', 'welfare', 'drugs',
                     'influence', 'gun', 'caution', 'auto', 'accident', 'reporting']

df_vocab = pd.read_csv('https://raw.githubusercontent.com/lupedeanda/test_repo/main/vocab.csv')
for word in df_vocab['Word']:
    final_list_of_words.append(word)

In [108]:
final_list_of_words = set(final_list_of_words)
#Converting to iterable elements

In [109]:
Full_Raw_DataFrame
#checking the dataframe (Original Raw Bill Data)

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
0,59869,30,AB1,5,Vetoed,20091011,Teachers: program of professional growth: conf...,An act to amend Section 44277 of the Education...,0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB1/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
1,59870,30,AB2,5,Vetoed,20091011,Individual health care coverage.,"An act to add Sections 1389.9, 1389.10, 1389.1...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB2/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
2,59871,30,AB3,5,Vetoed,20091011,Workforce development: Renewable Energy Workforce,"An act to amend Sections 14202, 14203, 14204, ...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB3/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
3,59872,30,AB4,1,Introduced,20081201,Constitutional convention: delegates.,An act relating to the Constitutional Convention.,0,NaN,20100202,From committee: Filed with the Chief Clerk pur...,https://legiscan.com/CA/bill/AB4/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
4,59873,30,AB5,4,Passed,20090629,Civil discovery: Electronic Discovery Act.,"An act to amend Sections 2016.020, 2031.010, 2...",0,NaN,20090629,"Chaptered by Secretary of State - Chapter 5, S...",https://legiscan.com/CA/bill/AB5/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33653,1385441,1791,SCA1,1,Introduced,20201207,Elections: referenda.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210407,Referred to Com. on E. & C.A.,https://legiscan.com/CA/bill/SCA1/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33654,1385541,1791,SCA2,1,Introduced,20201207,Public housing projects.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210429,From committee: Be adopted and re-refer to Com...,https://legiscan.com/CA/bill/SCA2/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33655,1385494,1791,SCA3,1,Introduced,20201207,Elections: recalls.,A resolution to propose to the people of the S...,1796,Senate Elections and Constitutional Amendments,20210407,Referred to Com. on E. & C.A.,https://legiscan.com/CA/bill/SCA3/2021,http://leginfo.legislature.ca.gov/faces/billSt...
33656,1455043,1791,SCA4,1,Introduced,20210217,Legislature: 2-year budget.,A resolution to propose to the people of the S...,476,Senate Budget and Fiscal Review,20210407,Referred to Coms. on B. & F.R. and E. & C.A.,https://legiscan.com/CA/bill/SCA4/2021,http://leginfo.legislature.ca.gov/faces/billSt...


In [110]:
Full_Raw_DataFrame_test = Full_Raw_DataFrame.copy()
#Making a copu of the original Dataframe to perform a string check test
index=0 #Using index to iterate through rows
keepcheck = False #Used to keep or remove the row
for row in Full_Raw_DataFrame_test['title']: #For each cell under title column
    row = str(row).lower() #lowercase the strings in the row
    rowcheck = row.split() #Split the string into muliple strings of words
    for string in rowcheck: #For each string in the row (which is a list of strings)
        string = string.replace(':','',) #Replacing : , . to empty characters
        string = string.replace('.','',) #Doing the same for .
        string = string.replace(',','',) #Doing the same for ,
        for term in final_list_of_words: #For each term within the referral list (law enforcement-related terms)
            if term == string: #If there is a match between both strings
                keepcheck=True #Then mark the document as KEEP

    if keepcheck == False: #If the document does not receive a KEEP pass
        row = '|' #Mark the row with | which will be removed as a whole
        Full_Raw_DataFrame_test.at[index,'title'] = row #Writing | into the row for removal
    keepcheck = False #Marking keepcheck as False again to prevent false positives

    index += 1 #Counting up within the index

In [112]:
Full_Raw_DataFrame_test = Full_Raw_DataFrame_test[~Full_Raw_DataFrame_test['title'].isin(['|'])] #Removing marked bills (non-relevant)
Full_Raw_DataFrame_test = Full_Raw_DataFrame_test.reset_index(drop=True) #Resetting index

#Full_Raw_DataFrame_test.to_csv("Bills_Dataframe_2009_2021.csv") #Writing csv
#Commenting-out this line to prevent overwriting

In [113]:
Full_Raw_DataFrame_test

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
0,61630,30,ABX32,2,Engrossed,20090112,VLF state parks fee: fuel taxes: emergency ser...,An act to add Section 8587.3 to the Government...,0,NaN,20091026,Died Concurrence pending.,https://legiscan.com/CA/bill/ABX32/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
1,59922,30,AB54,1,Introduced,20081204,State government: California Emergency Managem...,An act relating to the California Emergency Ma...,0,NaN,20100202,From committee: Filed with the Chief Clerk pur...,https://legiscan.com/CA/bill/AB54/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
2,59931,30,AB63,4,Passed,20090806,Service contracts: retailers.,An act to add Section 1794.45 to the Civil Cod...,0,NaN,20090806,"Chaptered by Secretary of State - Chapter 74, ...",https://legiscan.com/CA/bill/AB63/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
3,61766,30,ABX76,1,Introduced,20091029,Water diversion and use: reporting.,"An act to amend Sections 5100, 5101, 5103, and...",0,NaN,20091104,From committee without further action.,https://legiscan.com/CA/bill/ABX76/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
4,59959,30,AB91,4,Passed,20091011,Vehicles: driving under the influence (DUI): i...,"An act to amend Sections 13386 and 23576 of, a...",0,NaN,20091011,"Chaptered by Secretary of State - Chapter 217,...",https://legiscan.com/CA/bill/AB91/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,1459281,1791,SB792,1,Introduced,20210219,Sales and use tax: retailers: reporting.,An act to add Section 7055.5 to the Revenue an...,475,Senate Appropriations,20210514,Set for hearing May 20.,https://legiscan.com/CA/bill/SB792/2021,http://leginfo.legislature.ca.gov/faces/billSt...
1807,1459293,1791,SB804,1,Introduced,20210219,California Conservation Corps: forestry traini...,"An act to amend Section 14424 of, and to add S...",475,Senate Appropriations,20210514,Set for hearing May 20.,https://legiscan.com/CA/bill/SB804/2021,http://leginfo.legislature.ca.gov/faces/billSt...
1808,1388850,1791,SCR5,1,Introduced,20201222,State of emergency: COVID-19: termination.,Relative to terminating the state of emergency...,479,Senate Governmental Organization,20210210,Re-referred to Com. on G.O.,https://legiscan.com/CA/bill/SCR5/2021,http://leginfo.legislature.ca.gov/faces/billSt...
1809,1422172,1791,SCR7,2,Engrossed,20210510,"CHP Officer Andre Maurice Moye, Jr. Memorial F...",Relative to the CHP Officer Andre Maurice Moye...,471,Assembly Transportation,20210513,Referred to Com. on TRANS.,https://legiscan.com/CA/bill/SCR7/2021,http://leginfo.legislature.ca.gov/faces/billSt...


In [96]:
#Full_Raw_DataFrame_test #checking dataframe
#Method is successful, now will create function to run content analysis of each bill and append a new column
#Column will have single word phrase which will then be matched to the string check loop above

#Creating a copy once again to attempt a webscrape content analysis of each bill within this dataframe
Full_Raw_DataFrame_test = Full_Raw_DataFrame.copy()

### The Hard Part (Webscraping)
- This is the part I've been putting off for far too long, and now I have to approach it
- I will hopefully webscrape the bills individually using the list of bills I currently have (the content of each bill)
- Bill content will be temporarily stored as a string and have stopwords removed along with non-regular characters
- String will be incorporated into a table (all strings will end up here)
- TF-IDF will be applied to each bill within the table in order to create an accurate determination of the context of each bill
- New column of frequent/weighted terms will be created using this analysis. Appended to master dataframe
- Column can now be used to apply loop above in order to effectively "weed out" bills that do not apply to law enforcement
- Bills Dataset is successfully cleaned!

In [61]:
#I finally had a breakthrough!!! I can search up bills directly using the bill number within the CA legislature bill site
#This is much better than using the urls given by the legiscan website and better than scouring the search parameters
#as a whole for a dataframe!


In [85]:
#Post parameters are inputted in as a dictionary, but they look a bit different when viewing through web inspector
url = ('https://leginfo.legislature.ca.gov/faces/billNavClient.xhtml?bill_id=' + '20192020' + '0' + 'SCA3')
#Format Looks to be: bill_id = 'Years Of Session (20212022)' + 0 + 'Bill Number'
#url = ('https://leginfo.legislature.ca.gov/faces/billNavClient.xhtml?bill_id=' + '20212022' + '0' + 'AB4')
response = requests.post(url) #Redoing the requests (receving and saving information after sending request) this time with the post parameters included
soup = BeautifulSoup(response.content, 'html.parser') #Creating the new "soup", html parsed using Beautiful Soup

In [86]:
soup.find_all(id="bill_all")
#IT WORKS YES!

[<div align="justify" id="bill_all"><div id="about"><br clear="all"/><div align="center"><table align="center" cellspacing="0" style="border-collapse: collapse"><tbody><tr><td align="center"></td></tr></tbody></table></div><br clear="all"/><div align="center"><font size="3" style="FONT-FAMILY: MinionPC, Arial Unicode MS">
                     CALIFORNIA LEGISLATURE—
                     2019–2020 REGULAR SESSION</font></div><br clear="all"/><span id="bill_house_num"><div style="float: left; width:50%; font-weight: bold; "><span style="text-transform: uppercase; text-align:left;width:50%;"><b>
                 Senate Constitutional Amendment
               </b></span></div><div style="float:right; width:50%; text-align:right;"><span style="text-transform: uppercase;"><b>No. 3</b></span></div></span><hr/><br clear="all"/><div align="center"><table align="center" cellspacing="0" style="border-collapse: collapse"><tbody><tr><td align="center" id="bill_authors"><span style="font-weight:bold

#### Now Applying This To All Bills as a test
* Changes to read from url = Bill Year Session + 0 + Bill Number
* Example: X = 20212022 + 0 + AB4
    * Will look like *202120220AB4* in the string
* Will need to quickly convert the single dates into two year timeframes

In [97]:
#Stripping unneeded integers from the dates (we only need the year)
templist = []
for row in Full_Raw_DataFrame_test["status_date"]:
    templist.append(row[0:4:1])

index = 0
#Converting years to 2-years
for i in templist:
    if int(i) == 2007 or int(i) == 2008:
        templist[index] = '20072008'
    elif int(i) == 2009 or int(i) == 2010:
        templist[index] = '20092010'
    elif int(i) == 2011 or int(i) == 2012:
        templist[index] = '20112012'
    elif int(i) == 2013 or int(i) == 2014:
        templist[index] = '20132014'
    elif int(i) == 2015 or int(i) == 2016:
        templist[index] = '20152016'
    elif int(i) == 2017 or int(i) == 2018:
        templist[index] = '20172018'
    elif int(i) == 2019 or int(i) == 2020:
        templist[index] = '20192020'
    elif int(i) == 2021 or int(i) == 2022:
        templist[index] = '20212022'
    index += 1
#Creating new column
Full_Raw_DataFrame_test['session_years'] = templist
Full_Raw_DataFrame_test.head(5) #Checking Dataframe

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link,session_years
0,59869,30,AB1,5,Vetoed,20091011,Teachers: program of professional growth: conf...,An act to amend Section 44277 of the Education...,0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB1/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,20092010
1,59870,30,AB2,5,Vetoed,20091011,Individual health care coverage.,"An act to add Sections 1389.9, 1389.10, 1389.1...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB2/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,20092010
2,59871,30,AB3,5,Vetoed,20091011,Workforce development: Renewable Energy Workforce,"An act to amend Sections 14202, 14203, 14204, ...",0,NaN,20100114,Consideration of Governor's veto stricken from...,https://legiscan.com/CA/bill/AB3/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,20092010
3,59872,30,AB4,1,Introduced,20081201,Constitutional convention: delegates.,An act relating to the Constitutional Convention.,0,NaN,20100202,From committee: Filed with the Chief Clerk pur...,https://legiscan.com/CA/bill/AB4/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,20072008
4,59873,30,AB5,4,Passed,20090629,Civil discovery: Electronic Discovery Act.,"An act to amend Sections 2016.020, 2031.010, 2...",0,NaN,20090629,"Chaptered by Secretary of State - Chapter 5, S...",https://legiscan.com/CA/bill/AB5/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,20092010


#ONLY RUN THIS EVERY 10ish minutes. It is extremely RAM and CPU intensive and will either do these things
#Best case scenario: Give you a RAM error
#Second Best Case: Slow or Freeze Python
#Absolute Worst Case: Get you IP Blocked from the Legislature Website

index = 0
#session_years + 0 + bill_number
templist = [] #Resetting list
#Iterating and scraping bill content into the list
for row in Full_Raw_DataFrame_test.iterrows():
    #print(str(Full_Raw_DataFrame_test.iloc[index]['session_years']) + '0' + str(Full_Raw_DataFrame_test.iloc[index]['bill_number']))
    #Applying BeautifulSoup url scrape

    url = ('https://leginfo.legislature.ca.gov/faces/billNavClient.xhtml?bill_id=' + str(Full_Raw_DataFrame_test.iloc[index]['session_years']) + '0' + str(Full_Raw_DataFrame_test.iloc[index]['bill_number']))
    response = requests.post(url) #Requesting webpage using url
    soup = BeautifulSoup(response.content, 'html.parser') #Using BeautifulSoup to go through content
    templist.append(soup.find_all(id="bill_all")) #Using BeautifulSoup to look for the bill text object and appending


    index += 1

#I will only run this again once I've figured out how to parse and pull only the text from the objects
#Update: aborted due the the sheer amount of RAM needed to run. This is better left to a text of data scientists and a supercomputer, as
#my macbook air and myself simple can't go any further from this point

In [101]:
#Will need to run on datahub or colab to get the entire list filled (aka all of the bills)
#It works, but only if I have more RAM to store the finished list
#Could possibly optimize by cleaning each scraped bill and performing term-search or
#context analysis prior to writing to a list (saves on memory)

[<div align="justify" id="bill_all"><div id="about"><br clear="all"/><div align="center"><table align="center" cellspacing="0" style="border-collapse: collapse"><tbody><tr><td align="center"><span style=" text-transform: uppercase; font-size: 1em">
                 Amended
                IN </span><span style=" text-transform: uppercase; font-size: 1em">
                 Assembly
               </span> <span style="text-transform: uppercase">March 31, 2009</span><br/></td></tr></tbody><tbody><tr><td align="center"></td></tr></tbody></table></div><br clear="all"/><div align="center"><font size="3" style="FONT-FAMILY: MinionPC, Arial Unicode MS">
                     CALIFORNIA LEGISLATURE—
                     2009–2010 REGULAR SESSION</font></div><br clear="all"/><span id="bill_house_num"><div style="float: left; width:50%; font-weight: bold; "><span style="text-transform: uppercase; text-align:left;width:50%;"><b>
                 Assembly Bill
               </b></span></div><div st